In [16]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"

In [19]:
from transformers import AutoTokenizer, AutoModel
import torch
torch.cuda.set_device(2)
torch.cuda.empty_cache()
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True).half().cuda()
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)

model = model.eval()

response, history = model.chat(tokenizer, "你有什么要对我说的吗", history=[])
print(response)

Loading checkpoint shards: 100%|██████████| 7/7 [00:11<00:00,  1.62s/it]


如果你有任何问题或需要帮助,请随时告诉我。我很乐意为你提供帮助。


In [7]:
! pip install goto-statement

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.


In [20]:
response, history = model.chat(tokenizer, "请判断如下句子是否是病句，如果是，请判断错误的类别并且返回修改后的结果。句子： 做科学上的事实判断，不同的人从不同的视角、不同的方式为出发点，终究可以获得一致的结论，达成某种程度的一致的意见。", history=[])
print(response)

句子本身没有明显的语法或语义错误,可以被视为健康的句子。


In [ ]:
import csv
import json

jsonfile=open('resp.json','w',encoding='gb2312')
count=1
notwritten=[]
with open('dataset.csv', 'r',encoding='gb2312') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        text=row[1]
        # print(text)
        with open('prompt.txt','r',encoding='utf-8')as f:
            prompt=f.read()
            # print(prompt)
            prompt=prompt+f"\n#\n{text}\n#"
            # print(prompt)
            response, history = model.chat(tokenizer, prompt, history=[])
            # print(response)
            try:
                respjson=json.loads(response)
                
            except:
                print('not written!')
                notwritten.append(count)
                pass
            json.dump(respjson,jsonfile)
            jsonfile.write(',')
            print(count)
            count+=1
    jsonfile.close()
        

In [23]:
jsonfile.close()

In [4]:
import json
with open('resp.json','r',encoding='utf-8')as f:
    data=json.load(f)
with open('resp.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [23]:
with open('resp.csv','w',encoding='utf-8') as f:
    writer=csv.writer(f)
    writer.writerow(data[0].keys())
    for item in data:
        writer.writerow(item.values())

modify&test:

In [12]:
import pandas as pd
import csv
content=[]
with open('resp.csv', 'r') as f:
    reader = csv.reader(f)
    # print(type(reader))
    header = next(reader) # 读取表头
    num_fields = len(header) # 字段数目
    content.append(header)
    for row in reader:
        # print(type(row))
        if len(row) != num_fields:
            print(f"Error: Expected {num_fields} fields in line {reader.line_num}, saw {len(row)}")
            del row[2]
            
            print('line ',reader.line_num,'has been modified!')
        row[0]=row[0].strip()
        row[1]=row[1].strip()
        row[2]=row[2].strip()
        content.append(row)
with open('resp.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(content)


transform encoding

In [2]:
import codecs
import csv
with open('dataset.csv', 'r', encoding='gb2312') as f:
    reader = csv.reader(f)
    rows=[row for row in reader]

with codecs.open('dataset.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(rows)


UnicodeDecodeError: 'gb2312' codec can't decode byte 0xe5 in position 6: illegal multibyte sequence

In [6]:
rows=[]
with open('dataset.csv', 'r') as f:
    reader=csv.reader(f)
    for row in reader:
        row[1]=row[1].strip()
        row[2]=row[2].strip()
        rows.append(row)
with open('dataset.csv', 'w') as f:
    writer= csv.writer(f)
    writer.writerows(rows)

In [13]:
import pandas as pd
df1=pd.read_csv('dataset.csv',low_memory=False)
print(df1)
# for i in df1:
#     print(i)
# print(df1.keys)
df2=pd.read_csv('resp.csv')
print(df2)
# for i in df2:
    # print(i)
df=pd.merge(df2,df1,on="病句",how='left')
df.to_csv('test.csv', encoding="utf_8_sig")

                                                     正确句子  \
0       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...   
1       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...   
2       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...   
3       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...   
4       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...   
...                                                   ...   
222609         "[20]（p.39）但随着研究的不断深入，学者们得出的结论也逐渐趋向比较客观和公允   
222610         "[20]（p.39）但随着研究的不断深入，学者们得出的结论也逐渐趋向比较客观和公允   
222611                      "、"白身"，把缺乏锻炼、阅历不深的文人称作"白面书生"等   
222612                      "、"白身"，把缺乏锻炼、阅历不深的文人称作"白面书生"等   
222613                      "、"白身"，把缺乏锻炼、阅历不深的文人称作"白面书生"等   

                                                       病句  语病类型  Unnamed: 3  \
0       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...  成分残缺         NaN   
1       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...  成分残缺         NaN   
2       做哲学上的价值判断，每个人都是根据自己的标准

莫名其妙就能跑了。。。

In [14]:
with open('test.csv','r')as f:
    judge_wrong=0
    modify_wrong=0
    tot_num=0
    reader = csv.reader(f)
    header=next(reader)
    for row in reader:
        if row[2]!=row[5]:
            judge_wrong+=1
        if row[3]!=row[4]:
            modify_wrong+=1
        tot_num+=1
jug_acc=1-judge_wrong/tot_num
mod_acc=1-modify_wrong/tot_num
print("judging accuracy:",jug_acc)
print("modifying cohesion:",mod_acc)
        

judging accuracy: 0.044487427466150864
modifying cohesion: 0.0
